In [ ]:
import pandas as pd
import numpy as np
import wrds
from functions import *
import os

os.makedirs(name=raw_data_path,exist_ok=True)
os.makedirs(name=processed_data_path, exist_ok=True)
conn = wrds.Connection()

一次性提取出所有需要的数据，避免重复提取

#### 1. Raw Data for accounting_60_hxz.ipynb

In [ ]:
comp = conn.raw_sql("""
                    /*header info*/
                    select c.gvkey, f.cusip, f.datadate, f.fyear, c.cik, substr(c.sic,1,2) as sic2, c.sic, c.naics,
                    
                    /*firm variables*/
                    /*income statement*/
                    f.sale, f.revt, f.cogs, f.xsga, f.dp, f.xrd, f.xad, f.ib, f.ebitda,
                    f.ebit, f.nopi, f.spi, f.pi, f.txp, f.ni, f.txfed, f.txfo, f.txt, f.xint,
                    
                    /*CF statement and others*/
                    f.capx, f.oancf, f.dvt, f.ob, f.gdwlia, f.gdwlip, f.gwo, f.mib, f.oiadp, f.ivao,
                    
                    /*assets*/
                    f.rect, f.act, f.che, f.ppegt, f.invt, f.at, f.aco, f.intan, f.ao, f.ppent, f.gdwl, f.fatb, f.fatl,
                    
                    /*liabilities*/
                    f.lct, f.dlc, f.dltt, f.lt, f.dm, f.dcvt, f.cshrc, 
                    f.dcpstk, f.pstk, f.ap, f.lco, f.lo, f.drc, f.drlt, f.txdi,
                    
                    /*equity and other*/
                    f.ceq, f.scstkc, f.emp, f.csho, f.seq, f.txditc, f.pstkrv, f.pstkl, f.np, f.txdc, f.dpc, f.ajex,
                    
                    /*market*/
                    abs(f.prcc_f) as prcc_f
                    
                    from comp.funda as f
                    left join comp.company as c
                    on f.gvkey = c.gvkey
                    
                    /*get consolidated, standardized, industrial format statements*/
                    where f.indfmt = 'INDL' 
                    and f.datafmt = 'STD'
                    and f.popsrc = 'D'
                    and f.consol = 'C'
                    and f.datadate >= '01/01/2016'
                    """)
comp.to_feather(os.path.join(raw_data_path, "comp.ftr"))

In [ ]:
crsp = conn.raw_sql("""
                      select a.prc, a.ret, a.retx, a.shrout, a.vol, a.cfacpr, a.cfacshr, a.date, a.permno, a.permco,
                      b.ticker, b.ncusip, b.shrcd, b.exchcd
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date >= '01/01/2016'
                      and b.exchcd between 1 and 3
                      """)
crsp.to_feather(os.path.join(raw_data_path,"crsp.ftr"))

In [ ]:
comp_q = conn.raw_sql("""
            /*header info*/
            select c.gvkey, f.cusip, f.datadate, f.fyearq,  substr(c.sic,1,2) as sic2, c.sic, f.fqtr, f.rdq,

            /*income statement*/
            f.ibq, f.saleq, f.txtq, f.revtq, f.cogsq, f.xsgaq, f.revty, f.cogsy, f.saley,

            /*balance sheet items*/
            f.atq, f.actq, f.cheq, f.lctq, f.dlcq, f.ppentq, f.ppegtq,

            /*others*/
            abs(f.prccq) as prccq, abs(f.prccq)*f.cshoq as mveq_f, f.ceqq, f.seqq, f.pstkq, f.ltq,
            f.pstkrq, f.gdwlq, f.intanq, f.mibq, f.oiadpq, f.ivaoq,
            
            /* v3 my formula add*/
            f.ajexq, f.cshoq, f.txditcq, f.npq, f.xrdy, f.xrdq, f.dpq, f.xintq, f.invtq, f.scstkcy, f.niq,
            f.oancfy, f.dlttq, f.rectq, f.acoq, f.apq, f.lcoq, f.loq, f.aoq

            from comp.fundq as f
            left join comp.company as c
            on f.gvkey = c.gvkey

            /*get consolidated, standardized, industrial format statements*/
            where f.indfmt = 'INDL' 
            and f.datafmt = 'STD'
            and f.popsrc = 'D'
            and f.consol = 'C'
            and f.datadate >= '01/01/2016'
            """)
comp_q.to_feather(os.path.join(raw_data_path, "comp_q.ftr"))

In [ ]:
crsp_mom = conn.raw_sql("""
                select permno, date, ret, retx, prc, shrout, vol
                from crsp.msf
                where date >= '01/01/2016'
                """)
crsp_mom.to_feather(os.path.join(raw_data_path, "crsp_mom.ftr"))

In [ ]:
dlret = conn.raw_sql("""
                     select permno, dlret, dlstdt 
                     from crsp.msedelist
                     """)
dlret.to_feather(os.path.join(raw_data_path, "dlret.ftr"))

In [ ]:
ccm_sql = """
            select gvkey, lpermno as permno, linktype, linkprim, 
            linkdt, linkenddt
            from crsp.ccmxpf_linktable
            where substr(linktype,1,1)='L'
            and (linkprim ='C' or linkprim='P')
        """
        
ccm = conn.raw_sql(ccm_sql)
ccm.to_feather(os.path.join(raw_data_path, "raw_data"))

#### 2. Raw Data for beta.ipynb(rvar_capm,ipynb, rvar_ff3.ipynb, rvar_mean.ipynb)

In [ ]:
# CRSP Block
crsp_beta = conn.raw_sql("""
                    select a.permno, a.date, a.ret, b.rf, b.mktrf, b.smb, b.hml
                    from crsp.dsf as a
                    left join ff.factors_daily as b
                    on a.date=b.date
                    where a.date > '01/01/2016'
                    """)
crsp_beta.to_feather(os.path.join(raw_data_path, "crsp_beta.ftr"))

#### 3.Raw Data for ill.ipynb(market_d.ipynb)

In [ ]:
# CRSP Block
crsp = conn.raw_sql("""
                    select a.permno, a.date, a.ret, a.vol, a.prc
                    from crsp.dsf as a
                    where a.date > '01/01/2016'
                    """)

crsp.to_feather(os.path.join(raw_data_path, "ill_crsp.ftr"))

#### 4. Raw Data for sue.ipynb

In [ ]:
comp = conn.raw_sql("""
                        select gvkey, datadate, fyearq, fqtr, epspxq, ajexq
                        from comp.fundq
                        where indfmt = 'INDL' 
                        and datafmt = 'STD'
                        and popsrc = 'D'
                        and consol = 'C'
                        and datadate >= '01/01/2016'
                        """)

comp['datadate'] = pd.to_datetime(comp['datadate'])
comp.to_feather(os.path.join(raw_data_path, "sue_comp.ftr"))

In [ ]:
ccm = conn.raw_sql("""
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where linktype in ('LU', 'LC')
                  """)
ccm.to_feather(os.path.join(raw_data_path, "sue_ccm.ftr"))

In [ ]:
# populate the quarterly sue to monthly
crsp_msf = conn.raw_sql("""
                        select distinct date
                        from crsp.msf
                        where date >= '01/01/2016'
                        """)
crsp_msf.to_feather(os.path.join(raw_data_path, "sue_crsp_msp.ftr"))

#### 5. Raw Data for std_dolvol.ipynb(std_turn.ipynb, zerotrade.ipynb)

In [ ]:
# CRSP Block
crsp = conn.raw_sql("""
                    select a.permno, a.date, a.vol, a.prc, a.shrout
                    from crsp.dsf as a
                    where a.date > '01/01/2016'
                    """)
crsp.to_feather(os.path.join(raw_data_path, "std_dolvol_crsp.ftr"))

#### 6.Raw Data for abr.ipynb

In [ ]:
comp = conn.raw_sql("""
                    select gvkey, datadate, rdq, fyearq, fqtr
                    from comp.fundq
                    where indfmt = 'INDL' 
                   and datafmt = 'STD'
                   and popsrc = 'D'
                    and consol = 'C'
                    and datadate >= '01/01/2016'                    """)
comp = pd.to_feather("raw_data/abr_comp.ftr")

In [ ]:
ccm = conn.raw_sql("""
                 select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where linktype in ('LU', 'LC')
                  """)

ccm = pd.to_feather("raw_data/abr_ccm.ftr")

In [ ]:
crsp_dsi = conn.raw_sql("""
                        select distinct date
                        from crsp.dsi
                        where date >= '01/01/2016'
                        """)
crsp_dsi = pd.to_feather("raw_data/abr_crsp_dsi.ftr")

In [ ]:
crsp_d = conn.raw_sql("""
                      select a.prc, a.ret, a.shrout, a.vol, a.cfacpr, a.cfacshr, a.permno, a.permco, a.date,
                      b.siccd, b.ncusip, b.shrcd, b.exchcd
                      from crsp.dsf as a
                      left join crsp.dsenames as b
                      on a.permno=b.permno
                     and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date >= '01/01/2016'
                     and b.exchcd between 1 and 3
                      and b.shrcd in (10,11)
                      """)
crsp_d = pd.to_feather("raw_data/abr_crsp_d.ftr")